# Specifying a reaction network
$$
\def\n{\mathbf{n}}
\def\x{\mathbf{x}}
\def\N{\mathbb{\mathbb{N}}}
\def\X{\mathbb{X}}
\def\NX{\mathbb{\N_0^\X}}
\def\C{\mathcal{C}}
\def\Jc{\mathcal{J}_c}
\def\DM{\Delta M_{c,j}}
\newcommand\diff{\mathop{}\!\mathrm{d}}
\def\Xc{\mathbf{X}_c}
\def\Yc{\mathbf{Y}_c}
\newcommand{\muset}[1]{\dot{\{}#1\dot{\}}}
$$

In [1]:
# initialize sympy printing (for latex output)
from sympy import init_printing
init_printing()

# import functions and classes for compartment models
from compartor import *

## Basics

The left-hand-side and right-hand-site of transitions are expressions involving zero or more `Compartments`.

To define a compartment $[\x]$, first create a content variable $\x \in \N_0^D$, then define a compartment $[\x]$ with content $\x$.

In [2]:
x = Content('x')
Compartment(x)

Content variables are $D$-dimensional. In equations, `x[d]` can be used to denote the copy number of chemical species `d`.

Next, we can build a stoichiometric equation using the `Transition` class, which takes two parameter: the reactants expression and the products expression. For empty reactants or products, use `EmptySet`.
The following defines the "Exit" transition from the nested birth-death process example.

In [3]:
transition_E = Transition(Compartment(x), EmptySet, name='E')
display(transition_E)

`name="E"` is an optional parameter, defining the subscript used for the transition class. (It is only used for displaying equations, e.g., the $h_E$ over the arrow in the above example.)

To define a propensity for the transition class, we have to define a content-independent rate constant $k_c$ and a reactant tuning function $g_c$.

In [4]:
k_E = Constant('k_E')
g_E = 1

To specify the transition class with its propensity, use the constructor `TransitionClass(transition_c, k_c, g_c)`.

In [5]:
Exit = TransitionClass(transition_E, k_E, g_E)
display(Exit)

The optional 4th argument of `TransitionClass` specifies the outcome distribution $\pi_c$.
This must be defined, if content variables occur in products that do not occur in reactants.

The type of $\pi_c$ is `OutcomeDistribution`, which is a class comprising
* an expression or symbol to use for displaying $\pi_c$ in compound expressions
* a function `conditional_expectation` that takes an expression over reactants $\Xc$, and returns it's expectation over all product compartment variables.

There are generators for several predefined outcome distributions. If nothing is specified, as in the above "Exit" transition example, `OutcomeDistribution.identity()` is used. This has symbol $1$ and `conditional_expectation` returns the expression over reactants $\Xc$ as is. This is appropriate when $\Yc = \emptyset$ or, more precisely, if all content variables occurring in products already occur in reactants.

The other generators are
* `poisson()` which specifies a poisson distribution over a scalar, and
* `uniform()` which specifies a uniform distribution over a scalar.

For example, `poisson()` can be used to define an "Intake" transition class as follows:

In [6]:
from sympy import Symbol
y = Content('y')
transition_I = Transition(EmptySet, Compartment(y), name='I')
k_I = Constant('k_I')
g_I = 1
pi_I = OutcomeDistribution.poisson(
    Symbol("\pi_{Poiss}(y; \lambda)"),
    y[0],
    Symbol("\lambda", positive=True))
Intake = TransitionClass(transition_I, k_I, g_I, pi_I)
display(Intake)

When displaying a `TransitionClass`, the assembled propensity function is shown, where $w(\n;\Xc)$ is derived from the reactants in the stoichiometric equation.
The function `display_propensity_details()` displays the constituent expressions $k_c$, $g_c$, and $\pi_c$.

In [7]:
display_propensity_details(Exit)

To specify a reaction network, collect the transition classes in a list.
The function `display_transition_classes()` displays the equations and propensities of the network.

In [8]:
transitions = [Exit, Intake]
display_transition_classes(transitions)

Display.__str__: TODO

## Chemical Events

To specify chemical reactions, we need the `ContentChange()` class. It is constructed with $D$ (number of species) integer arguments, specifying the change (to a compartment variable) in each species.

For example, the "birth" transition from the nested birth-death process example can be defined as

In [9]:
x = Content('x')
Transition(Compartment(x), Compartment(x + ContentChange(1)), name='b')

Note, that `Compartment(...)` in the product encloses `x + ContentChange(1)`, so the `+` acts on the compartment content, rather than specifying a set of compartments.

Besides addition, also multiplication can be used inside compartment content equations, e.g.

In [10]:
Transition(Compartment(x), Compartment(2*x + ContentChange(1)))

## Compartment Events

To specify compartment events, simply construct (multi-)sets of compartments using `+`.

For example, a "Coagulation" transition can be specified as

In [11]:
x = Content('x')
y = Content('y')
Transition(Compartment(x) + Compartment(y), Compartment(x + y), name='C')

In principle, also multiplication can be used, e.g., `2*Compartment(x)` means "two compartments of content 'x'".
But the moment derivation mechanics is not ready for that yet...

## Example: nested birth-death process

As a complete example, here is the definition of nested birth-death process.

In [12]:
y = Content('y')
x = Content('x')

# Intake
transition_I = Transition(EmptySet, Compartment(y), name='I')
k_I = Constant('k_I')
g_I = 1
pi_I = OutcomeDistribution.poisson(
    Symbol("\pi_{Poiss}(y; \lambda)", positive=True),
    y[0],
    Symbol("\lambda", positive=True))
Intake = TransitionClass(transition_I, k_I, g_I, pi_I)

# Exit
transition_E = Transition(Compartment(x), EmptySet, name='E')
k_E = Constant('k_E')
g_E = 1
Exit = TransitionClass(transition_E, k_E, g_E)

# birth
transition_b = Transition(Compartment(x), Compartment(x + ContentChange(1)), name='b')
k_b = Constant('k_b')
g_b = 1
Birth = TransitionClass(transition_b, k_b, g_b)

# death
transition_d = Transition(Compartment(x), Compartment(x + ContentChange(-1)), name='d')
k_d = Constant('k_d')
g_d = x[0] # TODO x should be enough here, in case D=1.
Death = TransitionClass(transition_d, k_d, g_d)

transitions = [Intake, Exit, Birth, Death]
display_transition_classes(transitions)

Display.__str__: TODO